# Circuit VS_RL1

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import math
import dpsimpy

#%matplotlib widget

## DPsim EMT

In [ ]:
# DPsim EMT simulation
name = 'EMT_VS_RC1'

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 = dpsimpy.emt.SimNode('n1')
n2 = dpsimpy.emt.SimNode('n2')

# initialize node voltages as in modelica
n1.set_initial_voltage(complex(10, 0))
n2.set_initial_voltage(complex(6.43587, 2.02189) * math.sqrt(2))

# Components
vs = dpsimpy.emt.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0),f_src=50)
r1 = dpsimpy.emt.ph1.Resistor('r_1')
r1.set_parameters(R=1)
c1 = dpsimpy.emt.ph1.Capacitor('c_1')
c1.set_parameters(C=0.001)

# Connections
vs.connect([gnd, n1])
r1.connect([n1, n2])
c1.connect([n2, gnd])

# Define system topology
system = dpsimpy.SystemTopology(50, [gnd, n1, n2], [vs, r1, c1])

# Logging
logger = dpsimpy.Logger(name)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('r_1.i_intf', 'i_intf', r1)

sim = dpsimpy.Simulation(name)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_time_step(0.0001)
sim.set_final_time(0.1)
sim.add_logger(logger)

sim.run()

In [ ]:
# read EMT results
work_dir = 'logs/'
log_name = 'EMT_VS_RC1'
print(work_dir + log_name + '.csv')

ts_dpsim_emt = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
ts_dpsim_emt['n1.v'].label = 'v1 EMT'
ts_dpsim_emt['n2.v'].label = 'v2 EMT'
ts_dpsim_emt['r_1.i_intf'].label = 'i12 EMT'
pt.plot_timeseries(1, ts_dpsim_emt['n1.v'])
pt.plot_timeseries(1, ts_dpsim_emt['n2.v'])
pt.plot_timeseries(2, ts_dpsim_emt['r_1.i_intf'])

## DPsim DP

In [ ]:
# DPsim DP simulation
name = 'DP_VS_RC1'

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode('n1')
n2 = dpsimpy.dp.SimNode('n2')

# initialize node voltages as in simulink
n1.set_initial_voltage(complex(10, 0))
n2.set_initial_voltage(complex(6.43587, 2.02189) * math.sqrt(2))

# Components
vs = dpsimpy.dp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0))
r1 = dpsimpy.dp.ph1.Resistor('r_1')
r1.set_parameters(R=1)
c1 = dpsimpy.dp.ph1.Capacitor('c_1')
c1.set_parameters(C=0.001)

# Connections
vs.connect([gnd, n1])
r1.connect([n1, n2])
c1.connect([n2, gnd])

# Define system topology
system = dpsimpy.SystemTopology(50, [gnd, n1, n2], [vs, r1, c1])

# Logging
logger = dpsimpy.Logger(name)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('r_1.i_intf', 'i_intf', r1)

sim = dpsimpy.Simulation(name)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_time_step(0.0001)
sim.set_final_time(0.1)
sim.add_logger(logger)

sim.run()

In [ ]:
# read DPsim DP results
work_dir = 'logs/'
log_name = 'DP_VS_RC1'
print(work_dir + log_name + '.csv')

ts_dpsim_dp = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
# convert to emt
ts_dpsim_dp_emt = ts.frequency_shift_list(ts_dpsim_dp, 50)

In [ ]:
ts_dpsim_dp_emt['n1.v_shift'].label = 'v1 DP'
ts_dpsim_dp_emt['n2.v_shift'].label = 'v2 DP'
ts_dpsim_dp_emt['r_1.i_intf_shift'].label = 'i12 DP'
pt.plot_timeseries(1, ts_dpsim_dp_emt['n1.v_shift'])
pt.plot_timeseries(1, ts_dpsim_dp_emt['n2.v_shift'])
pt.plot_timeseries(2, ts_dpsim_dp_emt['r_1.i_intf_shift'])

## Simulink reference results

In [ ]:
# read Simulink log file
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/Modelica/BasicGrids/VS_RC1_Modelica.csv'
local_file = 'reference-results/VS_RC1_Modelica.csv'
urllib.request.urlretrieve(url, local_file) 

ts_mod = rt.read_timeseries_simulink(local_file)

In [ ]:
pt.set_timeseries_labels(ts_mod['v1'], 'v1 Mod')
pt.set_timeseries_labels(ts_mod['v2'], 'v2 Mod')
pt.set_timeseries_labels(ts_mod['i'], 'i12 Mod')
pt.plot_timeseries(1, ts_mod['v1'])
pt.plot_timeseries(1, ts_mod['v2'])
pt.plot_timeseries(2, ts_mod['i'])

## Comparison

In [ ]:
# plot v1
pt.plot_timeseries(1, ts_dpsim_emt['n1.v'])
pt.plot_timeseries(1, ts_dpsim_dp_emt['n1.v_shift'])
pt.plot_timeseries(1, ts_mod['v1'])
# plot v2
pt.plot_timeseries(2, ts_dpsim_emt['n2.v'])
pt.plot_timeseries(2, ts_dpsim_dp_emt['n2.v_shift'])
pt.plot_timeseries(2, ts_mod['v2'])
# plot i12
pt.plot_timeseries(3, ts_dpsim_emt['r_1.i_intf'])
pt.plot_timeseries(3, ts_dpsim_dp_emt['r_1.i_intf_shift'])
ts_sl_scale = ts_mod['i'].scale(-1)
ts_sl_scale.label = '-i Mod'
pt.plot_timeseries(3, ts_sl_scale)

In [ ]:
# calculate the RMSE between Simulink (ts_sl) and EMT (ts_dpsim_emt)
err_sl_emt = 0
err_sl_emt += ts.rmse(ts_mod['v1'], ts_dpsim_emt['n1.v'])
err_sl_emt += ts.rmse(ts_mod['v2'], ts_dpsim_emt['n2.v'])
err_sl_emt = err_sl_emt / 2
print("Total RMSE of Modelica reference and DPsim EMT: %g" % (err_sl_emt))

# calculate the RMSE between Simulink (ts_sl) and DP (ts_dpsim_dp_emt)
err_sl_dp = 0
err_sl_dp += ts.rmse(ts_mod['v1'], ts_dpsim_dp_emt['n1.v_shift'])
err_sl_dp += ts.rmse(ts_mod['v2'], ts_dpsim_dp_emt['n2.v_shift'])
err_sl_dp = err_sl_dp / 2
print("Total RMSE of Modelica reference and DPsim DP: %g" % (err_sl_dp))

In [ ]:
assert err_sl_emt < 0.062
assert err_sl_dp < 0.062